# Using optimizers

In [ ]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import numpy as np
np.random.seed(0)

In [ ]:
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn

mean, std = (0.5,), (0.5,)

# Create a transform and normalise data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                              ])

# Download FMNIST training dataset and load training data
trainset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download FMNIST test dataset and load test data
testset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 26421880/26421880 [00:00<00:00, 122663177.46it/s]


Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 5997232.95it/s]

Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 62838209.50it/s]


Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 1295122.18it/s]

Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw



In [12]:
class FMNIST(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 128)
    self.fc2 = nn.Linear(128,64)
    self.fc3 = nn.Linear(64,10)

  def forward(self, x):
    x = x.view(x.shape[0], -1)

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = F.log_softmax(x, dim=1)

    return x

In [ ]:
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

In [ ]:
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

In [ ]:
from torch import optim

# define criteria and optimizer
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
output = model(images)

# calculate loss
loss = criterion(output, labels)

# calcualte gradients
loss.backward()
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)


Initial weights :  Parameter containing:
tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0295,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0220, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0066,  0.0125,  ...,  0.0286,  0.0350, -0.0105]],
       requires_grad=True)
Initial weights gradient :  tensor([[-0.0030, -0.0030, -0.0030,  ..., -0.0030, -0.0030, -0.0030],
        [ 0.0022,  0.0022,  0.0022,  ...,  0.0024,  0.0022,  0.0022],
        [ 0.0002,  0.0002,  0.0002,  ...,  0.0002,  0.0002,  0.0002],
        ...,
        [ 0.0014,  0.0014,  0.0014,  ...,  0.0014,  0.0014,  0.0014],
        [ 0.0021,  0.0021,  0.0021,  ...,  0.0022,  0.0021,  0.0021],
        [ 0.0038,  0.0038,  0.0038,  ...,  0.0038,  0.0038,  0.0038]])


In [ ]:
# gradients associated with the first layer
model[0].weight.grad.shape

torch.Size([128, 784])

In [ ]:
# apply gradients
optimizer.step()

In [ ]:
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)

Initial weights :  Parameter containing:
tensor([[-0.0002,  0.0192, -0.0294,  ...,  0.0220,  0.0038,  0.0021],
        [-0.0198, -0.0150, -0.0105,  ..., -0.0203, -0.0060, -0.0300],
        [-0.0202,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0296,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0221, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0065,  0.0125,  ...,  0.0285,  0.0349, -0.0106]],
       requires_grad=True)
Initial weights gradient :  tensor([[-0.0030, -0.0030, -0.0030,  ..., -0.0030, -0.0030, -0.0030],
        [ 0.0022,  0.0022,  0.0022,  ...,  0.0024,  0.0022,  0.0022],
        [ 0.0002,  0.0002,  0.0002,  ...,  0.0002,  0.0002,  0.0002],
        ...,
        [ 0.0014,  0.0014,  0.0014,  ...,  0.0014,  0.0014,  0.0014],
        [ 0.0021,  0.0021,  0.0021,  ...,  0.0022,  0.0021,  0.0021],
        [ 0.0038,  0.0038,  0.0038,  ...,  0.0038,  0.0038,  0.0038]])


In [ ]:
# set gradients to zero as we want to calculate gradients w.r.t. loss of the next batch
optimizer.zero_grad()

In [ ]:
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)

Initial weights :  Parameter containing:
tensor([[-0.0002,  0.0192, -0.0294,  ...,  0.0220,  0.0038,  0.0021],
        [-0.0198, -0.0150, -0.0105,  ..., -0.0203, -0.0060, -0.0300],
        [-0.0202,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0296,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0221, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0065,  0.0125,  ...,  0.0285,  0.0349, -0.0106]],
       requires_grad=True)
Initial weights gradient :  None


In [15]:
model = FMNIST()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 3

for i in range(num_epochs):
    cum_loss = 0
    batch_n = 0

    for batch_n, (images, labels) in enumerate(trainloader):
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        cum_loss += loss.item()
        print(f'batch:{batch_n}, loss:{loss.item()}')


    print(f"Training loss: {cum_loss/len(trainloader)}")

batch:0, loss:2.3215713500976562
batch:1, loss:2.3074803352355957
batch:2, loss:2.3164830207824707
batch:3, loss:2.297412157058716
batch:4, loss:2.2926530838012695
batch:5, loss:2.2953672409057617
batch:6, loss:2.3005030155181885
batch:7, loss:2.2910714149475098
batch:8, loss:2.2748677730560303
batch:9, loss:2.2863876819610596
batch:10, loss:2.268754720687866
batch:11, loss:2.268362522125244
batch:12, loss:2.286766529083252
batch:13, loss:2.2703473567962646
batch:14, loss:2.2913095951080322
batch:15, loss:2.2611446380615234
batch:16, loss:2.2757766246795654
batch:17, loss:2.27260160446167
batch:18, loss:2.27754807472229
batch:19, loss:2.273784875869751
batch:20, loss:2.230733633041382
batch:21, loss:2.2553842067718506
batch:22, loss:2.2328648567199707
batch:23, loss:2.255995035171509
batch:24, loss:2.2610247135162354
batch:25, loss:2.240332841873169
batch:26, loss:2.235386848449707
batch:27, loss:2.2434561252593994
batch:28, loss:2.2051637172698975
batch:29, loss:2.2206692695617676
bat